In [1]:
import pandas as pd

# 1. Cargar predicciones
df_lgbm = pd.read_csv("predicciones_febrero2020_lgbm_lags12_deltas_combinado.csv")
df_xgb = pd.read_csv("predicciones_febrero2020_xgb_lags12_deltas_combinado.csv")
df_reg = pd.read_csv("predicciones_febrero2020_regresion_lags12_deltas_combinado.csv")

# 2. Renombrar columnas
df_lgbm.rename(columns={"tn_predicho": "lgbm"}, inplace=True)
df_xgb.rename(columns={"tn_predicho": "xgb"}, inplace=True)
df_reg.rename(columns={"tn_predicho": "regresion"}, inplace=True)

# 3. Merge por product_id
df_blend = df_lgbm.merge(df_xgb, on="product_id").merge(df_reg, on="product_id")

# 4. Crear blends
df_blend["blend_promedio"] = df_blend[["lgbm", "xgb", "regresion"]].mean(axis=1)
df_blend["blend_minimo"] = df_blend[["lgbm", "xgb", "regresion"]].min(axis=1)
df_blend["blend_ponderado"] = 0.9 * df_blend["blend_promedio"] + 0.1 * df_blend["blend_minimo"]

# 5. Guardar salidas
df_blend[["product_id", "blend_promedio"]].rename(columns={"blend_promedio": "tn_predicho"}).to_csv("blend_promedio.csv", index=False)
df_blend[["product_id", "blend_minimo"]].rename(columns={"blend_minimo": "tn_predicho"}).to_csv("blend_minimo.csv", index=False)
df_blend[["product_id", "blend_ponderado"]].rename(columns={"blend_ponderado": "tn_predicho"}).to_csv("blend_ponderado.csv", index=False)

# 6. Confirmar
print("✅ Archivos generados:")
print("📁 blend_promedio.csv")
print("📁 blend_minimo.csv")
print("📁 blend_ponderado.csv")


✅ Archivos generados:
📁 blend_promedio.csv
📁 blend_minimo.csv
📁 blend_ponderado.csv


In [ ]:
import pandas as pd
import numpy as np

# 1. Cargar predicciones
lgbm = pd.read_csv("predicciones_febrero2020_lgbm_agosto_cv.csv").rename(columns={"tn_predicho": "lgbm"})
xgb = pd.read_csv("predicciones_febrero2020_xgb_agosto_cv.csv").rename(columns={"tn_predicho": "xgb"})
reg = pd.read_csv("predicciones_febrero2020_regresion_agosto_cv.csv").rename(columns={"tn_predicho": "regresion"})

# 2. Unir
df = lgbm.merge(xgb, on="product_id").merge(reg, on="product_id")

# 3. Mínimo
df["blend_min"] = df[["lgbm", "xgb", "regresion"]].min(axis=1)

# 4. Máximo
df["blend_max"] = df[["lgbm", "xgb", "regresion"]].max(axis=1)

# 5. Promedio simple
df["blend_mean"] = df[["lgbm", "xgb", "regresion"]].mean(axis=1)

# 6. Ponderado (40% LGBM, 40% XGB, 20% Regresión)
df["blend_weighted"] = 0.4 * df["lgbm"] + 0.4 * df["xgb"] + 0.2 * df["regresion"]

# 7. Votación por cercanía (si difieren <10%, usar promedio; si no, usar mediana)
def blend_voto(row):
    vals = [row["lgbm"], row["xgb"], row["regresion"]]
    r = max(vals) - min(vals)
    if r < 0.1 * max(vals):  # diferencias chicas
        return np.mean(vals)
    else:
        return np.median(vals)

df["blend_voto"] = df.apply(blend_voto, axis=1)

# 8. Guardar archivos
df[["product_id", "blend_min"]].rename(columns={"blend_min": "tn_predicho"}).to_csv("blend_febrero2020_min.csv", index=False)
df[["product_id", "blend_max"]].rename(columns={"blend_max": "tn_predicho"}).to_csv("blend_febrero2020_max.csv", index=False)
df[["product_id", "blend_mean"]].rename(columns={"blend_mean": "tn_predicho"}).to_csv("blend_febrero2020_mean.csv", index=False)
df[["product_id", "blend_weighted"]].rename(columns={"blend_weighted": "tn_predicho"}).to_csv("blend_febrero2020_weighted.csv", index=False)
df[["product_id", "blend_voto"]].rename(columns={"blend_voto": "tn_predicho"}).to_csv("blend_febrero2020_voto.csv", index=False)

print("Blends generados:")
print("- blend_febrero2020_min.csv")
print("- blend_febrero2020_max.csv")
print("- blend_febrero2020_mean.csv")
print("- blend_febrero2020_weighted.csv")
print("- blend_febrero2020_voto.csv")


✅ Blends generados:
- blend_febrero2020_min.csv
- blend_febrero2020_max.csv
- blend_febrero2020_mean.csv
- blend_febrero2020_weighted.csv
- blend_febrero2020_voto.csv


In [5]:
import pandas as pd

# Cargar predicciones y MAEs
pred_lgbm = pd.read_csv("predicciones_febrero2020_lgbm_agosto_cv1.csv")
mae_lgbm = pd.read_csv("mae_lgbm_cv1.csv")
pred_xgb = pd.read_csv("predicciones_febrero2020_xgb_agosto_cv1.csv")
mae_xgb = pd.read_csv("mae_xgb_cv1.csv").rename(columns={"mae_cv": "mae_cv_xgb"})
pred_reg = pd.read_csv("predicciones_febrero2020_regresion_agosto_cv1.csv")
mae_reg = pd.read_csv("mae_ridge_cv1.csv").rename(columns={"mae_cv": "mae_cv_reg"})

# Unificar todos en un único DataFrame
predicciones = pd.DataFrame()
predicciones["product_id"] = pred_lgbm["product_id"]
predicciones["lgbm"] = pred_lgbm["tn_predicho"]
predicciones["xgb"] = pred_xgb["tn_predicho"]
predicciones["regresion"] = pred_reg["tn_predicho"]

predicciones = predicciones.merge(mae_lgbm, on="product_id", how="left")
predicciones = predicciones.merge(mae_xgb, on="product_id", how="left")
predicciones = predicciones.merge(mae_reg, on="product_id", how="left")

# Seleccionar el modelo con menor MAE para cada producto
def elegir_prediccion(row):
    maes = {
        "lgbm": row["mae_cv"],
        "xgb": row["mae_cv_xgb"],
        "regresion": row["mae_cv_reg"]
    }
    mejor_modelo = min(maes, key=maes.get)
    return row[mejor_modelo]

predicciones["tn_predicho"] = predicciones.apply(elegir_prediccion, axis=1)

# Resultado final
salida = predicciones[["product_id", "tn_predicho"]]
salida.to_csv("blend_febrero2020_cv1.csv", index=False)

print("Archivo final generado: blend_febrero2020_cv1.csv")

Archivo final generado: blend_febrero2020_cv1.csv
